In [ ]:
import numpy as np
import statsmodels.api as sm
import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, PercentChange, SimpleMovingAverage, BusinessDaysSincePreviousEvent, BusinessDaysUntilNextEvent, VWAP
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import factset
from quantopian.pipeline import factors, filters, classifiers
import alphalens
from scipy.stats.mstats import winsorize
from sklearn import preprocessing
from scipy.stats.mstats import gmean
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)

from quantopian.pipeline.data.factset.ownership import Form3AggregatedTrades
from quantopian.pipeline.data.factset.ownership import Form4and5AggregatedTrades


In [ ]:
sd = '2014-01-04'
ed = '2017-05-01'
                
insider_txns_form3_1d = Form3AggregatedTrades.slice(False, 1)
insider_txns_form4and5_1d = Form4and5AggregatedTrades.slice(False, 1)

insider_txns_form3_90d = Form3AggregatedTrades.slice(False, 90)
insider_txns_form4and5_90d = Form4and5AggregatedTrades.slice(False, 90)


#Get unique buyers/sellers    
filers_form3_1d = insider_txns_form3_1d.num_unique_filers
buyers_form4and5_1d = insider_txns_form4and5_1d.num_unique_buyers
sellers_form4and5_1d = insider_txns_form4and5_1d.num_unique_sellers

filers_form3_90d = insider_txns_form3_90d.num_unique_filers
buyers_form4and5_90d = insider_txns_form4and5_90d.num_unique_buyers
sellers_form4and5_90d = insider_txns_form4and5_90d.num_unique_sellers


class TxnsCount(CustomFactor):
    window_length = 3
    window_safe = True
    def compute(self, today, assets, out, b):  
        b=np.nan_to_num(b)
        m = b[-1]
        
        idx_zero = np.where(m == 0)[0]
        m[idx_zero] = b[-2][idx_zero]

        idx_zero = np.where(m == 0)[0]
        m[idx_zero] = b[-3][idx_zero]

        out[:] = m

                
base_universe = QTradableStocksUS()

f3_prev = BusinessDaysSincePreviousEvent(inputs=[insider_txns_form3_1d.asof_date])  
f5_prev = BusinessDaysSincePreviousEvent(inputs=[insider_txns_form4and5_1d.asof_date])  


class Asof(CustomFactor):
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, b):  
        out[:] = b[-1]

        
class TxnsCount(CustomFactor):
    window_length = 3
    window_safe = True
    def compute(self, today, assets, out, b):  
        b=np.nan_to_num(b)
        m = b[-1]
        
        idx_zero = np.where(m == 0)[0]
        m[idx_zero] = b[-2][idx_zero]

        idx_zero = np.where(m == 0)[0]
        m[idx_zero] = b[-3][idx_zero]

        out[:] = m
        

pipe = Pipeline(
    columns={
        
        'f3_prev_days': f3_prev,
        'f5_prev_days': f5_prev,
        
        'f3_asof': Asof(inputs=[insider_txns_form3_1d.asof_date]),
        'f5_asof': Asof(inputs=[insider_txns_form4and5_1d.asof_date]),
        
        'f3_buyers': TxnsCount(inputs=[filers_form3_1d]),
        'f5_buyers': TxnsCount(inputs=[buyers_form4and5_1d]),
        'f5_sellers': TxnsCount(inputs=[sellers_form4and5_1d]),
    },
    
    screen = base_universe & (filers_form3_90d.latest.notnull() | buyers_form4and5_90d.latest.notnull() | sellers_form4and5_90d.latest.notnull()),
)


output = run_pipeline(pipe, sd, ed) 

In [ ]:
output['f5_asof'] = pd.to_datetime(output['f5_asof'])
output['f3_asof'] = pd.to_datetime(output['f3_asof'])

print output.head(10)